# Computing ALFF
Imput data should not be band pass and standardize 

## <font color=#B2D732> <span style="background-color: #4424D6"> Imports

In [21]:
import sys, glob
main_dir="/cerebro/cerebro1/dataset/bmpd/derivatives/Aging_project/"

sys.path.append(main_dir + "/2025_brsc_aging_project/code/")
from connectivity.alff import ALFF
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nibabel as nib
import os.path
import json

from plotting import Plotting

# Set numpy to print only 2 decimal digits for neatness
np.set_printoptions(precision=2, suppress=True)

# bibliography:
# https://doi.org/10.1016/j.neuroimage.2009.09.037

%matplotlib inline
%load_ext autoreload
%autoreload 2
    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## <font color=#B2D732> <span style="background-color: #4424D6"> Load configuration

In [22]:
with open('../../config/analyses/brsc_functional.json') as config_file:
    config = json.load(config_file)

## <font color=#B2D732> <span style="background-color: #4424D6"> 1. ALFF computations

#### A. Create instance of ALFF class for further analyses

In [23]:
#redo with smoothed data to compare
structure='brain' #'spinalcord'
ana="alff"
alff = ALFF(config,IDs=config["participants_IDs_ALL"],structure=structure,analysis=ana) 

Output directory: /cerebro/cerebro1/dataset/bmpd/derivatives/Aging_project//figures/f02_functional_sc/preprint2025/alff/brain//alff/slow5/


#### B. Compute individual maps

In [27]:
fwhm=[3,3,3] if structure =="spinalcord" else [1,1,1]
alff_files,zalff_files=alff.compute_alff_maps(fwhm=fwhm,scaling_method="zscore",redo=False,n_jobs=4)

ALFF MAPS COMPUTATION
Overwrite old files: False
... Preparing data
... alff is Running
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:   24.5s
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:   24.5s
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:   24.5s
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:   24.5s
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   24.6s
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:   24.6s
[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed:   24.6s
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:   24.6s
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:   24.6s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   24.6s
[Parallel(n_jobs=4)]: Done  11 tasks      | elapsed:   24.6s
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:   24.6s
[Parallel(n_jobs=4)]: Done  13 tasks      | elapsed:   24.6s
[Parallel(n_jobs=4)]: Done  14 tasks      | 

/cerebro/cerebro1/dataset/bmpd/derivatives/Aging_project//2025_brsc_aging_project/code/connectivity/alff.py:370: RuntimeWarning: Mean of empty slice
  alff_map = np.sqrt(np.nanmean(power_spectrum[..., f_mask_bp], axis=-1))
/cerebro/cerebro1/dataset/bmpd/derivatives/Aging_project//2025_brsc_aging_project/code/connectivity/alff.py:370: RuntimeWarning: Mean of empty slice
  alff_map = np.sqrt(np.nanmean(power_spectrum[..., f_mask_bp], axis=-1))


[Parallel(n_jobs=4)]: Done  57 tasks      | elapsed:   51.7s
[Parallel(n_jobs=4)]: Done  58 tasks      | elapsed:   51.7s


/cerebro/cerebro1/dataset/bmpd/derivatives/Aging_project//2025_brsc_aging_project/code/connectivity/alff.py:370: RuntimeWarning: Mean of empty slice
  alff_map = np.sqrt(np.nanmean(power_spectrum[..., f_mask_bp], axis=-1))


[Parallel(n_jobs=4)]: Done  59 tasks      | elapsed:   52.2s


/cerebro/cerebro1/dataset/bmpd/derivatives/Aging_project//2025_brsc_aging_project/code/connectivity/alff.py:370: RuntimeWarning: Mean of empty slice
  alff_map = np.sqrt(np.nanmean(power_spectrum[..., f_mask_bp], axis=-1))


[Parallel(n_jobs=4)]: Done  60 tasks      | elapsed:   53.7s
[Parallel(n_jobs=4)]: Done  61 out of  67 | elapsed:  1.3min remaining:    7.9s
[Parallel(n_jobs=4)]: Done  62 out of  67 | elapsed:  1.4min remaining:    6.5s
[Parallel(n_jobs=4)]: Done  63 out of  67 | elapsed:  1.4min remaining:    5.2s
[Parallel(n_jobs=4)]: Done  64 out of  67 | elapsed:  1.4min remaining:    3.8s
[Parallel(n_jobs=4)]: Done  65 out of  67 | elapsed:  1.7min remaining:    3.2s
[Parallel(n_jobs=4)]: Done  67 out of  67 | elapsed:  1.7min finished
... Operation performed in 104.354 s
...DONE!


## <font color=#B2D732> <span style="background-color: #4424D6"> 2. ALFF ROI analyses

### A. Compute values per ROI
- within gm mask

In [28]:
alff_df={}

if structure =="brain":
    atlas={"schaefer200_cobra":"schaefer200_cobra"}
    template_dir=config["project_dir"] + "/templates/MNI/atlas/"
    tag_sup="/"
else:
    atlas={"levels":"_"}
    template_dir=config["project_dir"] + "/templates/PAM50/atlas/Frostell"
    tag_sup="28Parcels_7Networks/Frostell"

#---- Load image and label file
for tag_nb, tag in enumerate(atlas):
    atlas_f=glob.glob(template_dir+atlas[tag]+tag_sup+atlas[tag]+"*.nii.gz")[0]
    atlas_labels=glob.glob(template_dir+atlas[tag]+tag_sup+atlas[tag]+"*.txt")[0]
    labelslist=np.genfromtxt(atlas_labels, usecols=1,skip_header=1, dtype="S", delimiter="\t", encoding=None).astype(str)
    labelsnb=np.genfromtxt(atlas_labels, usecols=0,skip_header=1, dtype="S", delimiter="\t", encoding=None).astype(str)
    alff_df[tag]=alff.extract_alff_rois(input_f=alff_files,atlas_f=atlas_f,atlas_labels=labelslist,atlas_labels_nb=labelsnb,tag="_" + tag,redo=True,plot=False,verbose=0)



ALFF IN ROIS
Overwritting old files 
5.694963184160533e-09
7.353085712330303e-09
8.295898947220915e-09
3.5991696865654305e-09
5.381856277503411e-09
7.447818164665578e-09
6.045046932159594e-09
7.724019284254339e-09
7.501533879669161e-09
5.463487440238168e-09
7.579570551443895e-09
4.985083950575446e-09
6.949529676005652e-09
